In [165]:
# Import live code changes in
%load_ext autoreload
%autoreload 

from pathlib import Path
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
from sovereign.flood import build_basin_curves

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Set filepaths and provide data info

In [187]:
root = Path.cwd().parent # find project root
risk_basin_path = os.path.join(root, 'outputs', 'flood', 'risk', 'basins', 'risk_basins.csv')
copula_path = os.path.join(root, 'outputs', 'flood', 'dependence', 'copulas')
risk_data = pd.read_csv(risk_basin_path)
# Drop first "unnamed column"
risk_data = risk_data.iloc[:, 1:]
# Add AEP column
risk_data['AEP'] = 1 / risk_data['RP']
# Add a column converting current prorection level into AEP
risk_data['Pr_L_AEP'] = np.where(risk_data['Pr_L'] == 0, 0, 1 / risk_data['Pr_L']) # using numpy where avoids zero division errors
risk_data.reset_index(drop=True, inplace=True)

#### Build basin loss-probability curves

In [238]:
basin_curves: dict[int, BasinLossCurve] = build_basin_curves(risk_data)

#### Run a simulation

In [258]:
# USER CONFIG
adaptation_aep = 0.01 # 100-year flood protection
n_years = 10000 # number of years to simulate

In [259]:
# Prepare simulation
all_sectors = {comp.sector for curve in basin_curves.values() for comp in curve.components} # Find all sectors
sector_baseline_losses = {s: np.zeros(n_years) for s in all_sectors}
sector_adapted_losses = {s: np.zeros(n_years) for s in all_sectors}
gva_sectors = ["Agriculture", "Manufacturing", "Service"]
cap_sectors = ["Public", "Private"]
basin_ids = list(basin_curves.keys())
# Load precomputed copula basin dependence simulations
t_random_numbers = pd.read_parquet(os.path.join(copula_path, "t_random_numbers.parquet.gzip"))

In [260]:
# Run a simulation assuming complete independence between basins
seed = 0
rng = np.random.default_rng(seed)
for t in tqdm(range(n_years)):
    sector_year_baseline = {s: 0.0 for s in all_sectors}
    sector_year_adapted = {s: 0.0 for s in all_sectors}

    for basin_id in basin_ids:
        curve = basin_curves[basin_id]
        aep_event = rng.uniform(0.0, 1.0)

        for s  in all_sectors:
            bl = curve.loss_at_event_aep(aep_event, sector=s)
            ad = curve.loss_at_event_aep(aep_event, scenario="adaptation", adapted_protection_aep=adaptation_aep, sector=s)
            sector_year_baseline[s] += bl
            sector_year_adapted[s] += ad
    
    for s in all_sectors:
        sector_baseline_losses[s][t] = sector_year_baseline[s]
        sector_adapted_losses[s][t] = sector_year_adapted[s]

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:51<00:00, 192.88it/s]


In [261]:
# GVA losses (sum of GVA sectors)
gva_baseline_losses = sum(sector_baseline_losses[s] for s in gva_sectors)
gva_adapted_losses  = sum(sector_adapted_losses[s]  for s in gva_sectors)

# Capital stock losses (sum of capital sectors)
cap_baseline_losses = sum(sector_baseline_losses[s] for s in cap_sectors)
cap_adapted_losses  = sum(sector_adapted_losses[s]  for s in cap_sectors)

print("GVA AAL baseline:", gva_baseline_losses.mean())
print("GVA AAL adapted :", gva_adapted_losses.mean())
print("Cap AAL baseline:", cap_baseline_losses.mean())
print("Cap AAL adapted :", cap_adapted_losses.mean())

print("GVA AAL avoided :", gva_baseline_losses.mean() - gva_adapted_losses.mean())
print("Cap AAL avoided :", cap_baseline_losses.mean() - cap_adapted_losses.mean())

GVA AAL baseline: 4116280185.5787745
GVA AAL adapted : 2357230382.5223155
Cap AAL baseline: 12323210698.05182
Cap AAL adapted : 7332322426.717206
GVA AAL avoided : 1759049803.056459
Cap AAL avoided : 4990888271.334615


In [262]:
# Run a simulation with copula dependence between basins
for t in tqdm(range(n_years)):
    sector_year_baseline = {s: 0.0 for s in all_sectors}
    sector_year_adapted = {s: 0.0 for s in all_sectors}
    random_ns = t_random_numbers.loc[t] # extract random numbers for year t

    for basin_id in basin_ids:
        # TEMP DEBUG (need to re-run copulas as some basins not included)
        basin_str = str(int(basin_id))
        if basin_str not in random_ns:
            continue # skip
        curve = basin_curves[basin_id]
        aep_event = 1-random_ns[str(int(basin_id))] # may also be 1 - r need to check

        for s  in all_sectors:
            bl = curve.loss_at_event_aep(aep_event, sector=s)
            ad = curve.loss_at_event_aep(aep_event, scenario="adaptation", adapted_protection_aep=adaptation_aep, sector=s)
            sector_year_baseline[s] += bl
            sector_year_adapted[s] += ad
    
    for s in all_sectors:
        sector_baseline_losses[s][t] = sector_year_baseline[s]
        sector_adapted_losses[s][t] = sector_year_adapted[s]

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:59<00:00, 168.41it/s]


In [263]:
# GVA losses (sum of GVA sectors)
gva_baseline_losses = sum(sector_baseline_losses[s] for s in gva_sectors)
gva_adapted_losses  = sum(sector_adapted_losses[s]  for s in gva_sectors)

# Capital stock losses (sum of capital sectors)
cap_baseline_losses = sum(sector_baseline_losses[s] for s in cap_sectors)
cap_adapted_losses  = sum(sector_adapted_losses[s]  for s in cap_sectors)

print("GVA AAL baseline:", gva_baseline_losses.mean())
print("GVA AAL adapted :", gva_adapted_losses.mean())
print("Cap AAL baseline:", cap_baseline_losses.mean())
print("Cap AAL adapted :", cap_adapted_losses.mean())

print("GVA AAL avoided :", gva_baseline_losses.mean() - gva_adapted_losses.mean())
print("Cap AAL avoided :", cap_baseline_losses.mean() - cap_adapted_losses.mean())

GVA AAL baseline: 4072124144.573389
GVA AAL adapted : 2343216690.5359674
Cap AAL baseline: 12297111087.01352
Cap AAL adapted : 7334120299.928883
GVA AAL avoided : 1728907454.0374217
Cap AAL avoided : 4962990787.084637


#### DIGNAD Implementation

In [277]:
# User config and PREP
n_years = 10000
GDP = 500e9
# National GVA figures from DOSE
agr_GVA = 42880325598
man_GVA = 162659433017
ser_GVA = 316647741231

adaptation_aep = 0.01
TRADABLE_SHARES = {
    "Agriculture": 1.0,
    "Manufacturing": 0.7,
    "Service": 0.5,
}
PRIVATE_TRADABLE_CAP_SHARE = 50 # assume % share of private capital that belongs to tradable sectors

# Calculate tradable and nontradable output based on DOSE numbers and sector plits
tradable_output_baseline = (
    agr_GVA * TRADABLE_SHARES["Agriculture"] +
    man_GVA * TRADABLE_SHARES["Manufacturing"] +
    ser_GVA * TRADABLE_SHARES["Service"]
)

nontrad_output_baseline = (
    agr_GVA * (1 - TRADABLE_SHARES["Agriculture"]) +
    man_GVA * (1 - TRADABLE_SHARES["Manufacturing"]) +
    ser_GVA * (1 - TRADABLE_SHARES["Service"])
)

In [285]:
# Prepare sector list and storage
# discover all sectors from the curves
all_sectors = {c.sector for curve in basin_curves.values() for c in curve.components}
gva_sectors = ["Agriculture", "Manufacturing", "Service"]
cap_sectors = ["Public", "Private"]
# per-sector annual losses (monetary)
sector_baseline_losses = {s: np.zeros(n_years) for s in all_sectors}
sector_adapted_losses  = {s: np.zeros(n_years) for s in all_sectors}
# DIGNAD aggregate series (per year)
trad_output_loss_pct_baseline    = np.zeros(n_years)
trad_output_loss_pct_adapted     = np.zeros(n_years)
nontrad_output_loss_pct_baseline = np.zeros(n_years)
nontrad_output_loss_pct_adapted  = np.zeros(n_years)
private_cap_damage_pct_gdp_baseline  = np.zeros(n_years)
private_cap_damage_pct_gdp_adapted   = np.zeros(n_years)
public_cap_damage_pct_gdp_baseline   = np.zeros(n_years)
public_cap_damage_pct_gdp_adapted    = np.zeros(n_years)
tradable_cap_damage_share_baseline   = np.zeros(n_years)
tradable_cap_damage_share_adapted    = np.zeros(n_years)

In [286]:
for t in tqdm(range(n_years)):
    # sector totals for this simulated year
    sector_year_baseline = {s: 0.0 for s in all_sectors}
    sector_year_adapted  = {s: 0.0 for s in all_sectors}
    random_ns = t_random_numbers.loc[t] # extract random numbers for year t

    for basin_id in basin_ids:
        # TEMP DEBUG (need to re-run copulas as some basins not included)
        basin_str = str(int(basin_id))
        if basin_str not in random_ns:
            continue # skip
        curve = basin_curves[basin_id]
        aep_event = 1-random_ns[str(int(basin_id))] # may also be 1 - r need to check

        for s in all_sectors:
            # baseline
            bl = curve.loss_at_event_aep(
                aep_event,
                scenario="baseline",
                sector=s,
            )
            # adaptation
            ad = curve.loss_at_event_aep(
                aep_event,
                scenario="adaptation",
                adapted_protection_aep=adaptation_aep,
                sector=s,
            )
            sector_year_baseline[s] += bl
            sector_year_adapted[s]  += ad

    # store per-sector time series
    for s in all_sectors:
        sector_baseline_losses[s][t] = sector_year_baseline[s]
        sector_adapted_losses[s][t]  = sector_year_adapted[s]

    # ---- MAP TO DIGNAD AGGREGATES (BASELINE) ----
    ag_b   = sector_year_baseline.get("Agriculture", 0.0)
    man_b  = sector_year_baseline.get("Manufacturing", 0.0)
    serv_b = sector_year_baseline.get("Service", 0.0)
    priv_b = sector_year_baseline.get("Private", 0.0)
    pub_b  = sector_year_baseline.get("Public", 0.0)

    trad_out_b = (
        ag_b  * TRADABLE_SHARES["Agriculture"] +
        man_b * TRADABLE_SHARES["Manufacturing"] +
        serv_b* TRADABLE_SHARES["Service"]
    )
    nontrad_out_b = (
        ag_b  * (1 - TRADABLE_SHARES["Agriculture"]) +
        man_b * (1 - TRADABLE_SHARES["Manufacturing"]) +
        serv_b* (1 - TRADABLE_SHARES["Service"])
    )

    trad_output_loss_pct_baseline[t]    = 100 * trad_out_b    / tradable_output_baseline
    nontrad_output_loss_pct_baseline[t] = 100 * nontrad_out_b / nontrad_output_baseline
    private_cap_damage_pct_gdp_baseline[t]  = 100 * priv_b / GDP
    public_cap_damage_pct_gdp_baseline[t]   = 100 * pub_b  / GDP

    if priv_b > 0:
        tradable_cap_damage_share_baseline[t] = PRIVATE_TRADABLE_CAP_SHARE
    else:
        tradable_cap_damage_share_baseline[t] = np.nan  # or 0.0

    # ---- MAP TO DIGNAD AGGREGATES (ADAPTED) ----
    ag_a   = sector_year_adapted.get("Agriculture", 0.0)
    man_a  = sector_year_adapted.get("Manufacturing", 0.0)
    serv_a = sector_year_adapted.get("Service", 0.0)
    priv_a = sector_year_adapted.get("Private", 0.0)
    pub_a  = sector_year_adapted.get("Public", 0.0)

    trad_out_a = (
        ag_a  * TRADABLE_SHARES["Agriculture"] +
        man_a * TRADABLE_SHARES["Manufacturing"] +
        serv_a* TRADABLE_SHARES["Service"]
    )
    nontrad_out_a = (
        ag_a  * (1 - TRADABLE_SHARES["Agriculture"]) +
        man_a * (1 - TRADABLE_SHARES["Manufacturing"]) +
        serv_a* (1 - TRADABLE_SHARES["Service"])
    )

    trad_output_loss_pct_adapted[t]    = 100 * trad_out_a    / GDP
    nontrad_output_loss_pct_adapted[t] = 100 * nontrad_out_a / GDP
    private_cap_damage_pct_gdp_adapted[t]  = 100 * priv_a / GDP
    public_cap_damage_pct_gdp_adapted[t]   = 100 * pub_a  / GDP

    if priv_a > 0:
        tradable_cap_damage_share_adapted[t] = PRIVATE_TRADABLE_CAP_SHARE
    else:
        tradable_cap_damage_share_adapted[t] = np.nan


100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [01:10<00:00, 141.61it/s]


In [287]:
shocks = pd.DataFrame({
    "year_index": np.arange(n_years),
    "dY_T": trad_output_loss_pct_baseline,       # tradable output loss (% trad output)
    "dY_N": nontrad_output_loss_pct_baseline,    # non-trad output loss (% non-trad output)
    "dK_priv": private_cap_damage_pct_gdp_baseline,  # % GDP
    "dK_pub":  public_cap_damage_pct_gdp_baseline,   # % GDP
})

In [293]:
cols = ["dY_T", "dY_N", "dK_priv", "dK_pub"]
q = 0.999  # 99th percentile thresholds
thresholds = shocks[cols].quantile(q)
thresholds

dY_T       22.325263
dY_N       25.950726
dK_priv    42.398185
dK_pub     15.123066
Name: 0.999, dtype: float64

In [294]:
mask = np.ones(len(shocks), dtype=bool)
for c in cols:
    mask &= shocks[c] >= thresholds[c]

extreme_all_high = shocks[mask]
extreme_all_high

,year_index,dY_T,dY_N,dK_priv,dK_pub
1073,1073,23.984686,27.753205,45.189882,16.106309
1104,1104,23.109953,26.380166,43.596121,15.422342
3872,3872,25.885314,29.045270,47.940787,17.744415
4089,4089,22.702249,26.045535,42.428922,15.289534
4335,4335,25.138196,28.581100,47.093847,17.073515
5632,5632,25.013147,28.334446,45.609074,16.569101
7520,7520,23.534317,27.233926,44.418311,15.961600
8665,8665,23.481041,26.775625,43.990406,15.912869
